# <center>第四章 动态规划法</center>

## 4.1 基于状态值函数的策略评估

&emsp;&emsp;根据状态值函数的贝尔曼方程，可以通过联立线性方程组的方法，求解得每个状态在对应策略$\pi$下的状态。针对在状态$[i, j]$的扫地机器人，我们可以列出如下的贝尔曼方程：<br>
<center>$v_\pi(S_{ij})=\sum\pi(a|S_{ij})p(S_{ij},a,s')(r(S_{ij},a)+\gamma v_\pi(s'))$</center>

&emsp;&emsp;通常可以采取以下两种方式提前结束迭代：<br>
（1）直接设置迭代次数。只要达到预期的迭代次数，即可停止迭代；<br>
（2）设定较小的阈值（次优界限）$\theta$。当$|v_\iota(s)-v_(\iota-1)(s)|<\theta$时，停止迭代。比较两次迭代的状态值函数差的绝对值$\Delta$（或差的平方），当$\Delta$最大值小于阈值时，终止迭代。
下面使用的是第二种方法。<br><br>


&emsp;&emsp;迭代方法：<br>
（1）异步：即在相邻的两个迭代轮次$\iota$和$\iota-1$，保存同一组状态值函数$V(s)$。在$V(s)$中，存储两轮混合的函数值。因此在每次计算中，如果状态$s$的值函数已被更新，那么当用到$V(s)$时，就使用已经更新过的数据。评估过程中，中间结果与状态评估的先后此序密切相关。<br>
<center><img src='./第四章图片/图4.1.png' width='600'></center>
<center>算法4.1使用的迭代方法是异步计算方式</center>

（2）同步：即每一迭代轮次$\iota$，都保存相邻两轮的状态值函数：$V_\iota(s)$和$V_{\iota-1}(s)$。在计算$V_\iota(s)$过程中，使用的全部是上一轮的$V_{\iota-1}(s)$值。评估过程中，中间结果与状态评估的先后次序无关。


**例4.1** 设定策略评估的起始位置为左下角（如下图所示），即充电桩位置，按序号顺序进行评估。<br>
&emsp;&emsp;机器人在非终止状态（除位置 0、12、19）均采取等概率策略$\pi(a|s)=1/|A(s)|$，$|A(s)|$为当前状态$s$可采取的动作个数。扫地机器人最多可以采取{Up,Down,Left,Right}4个动作。到达充电桩位置时，r= +1；到达垃圾位置时， r=+3；撞到障碍物时，r=−10；其他情况，获得的奖赏 r 均为0。折扣系数$\gamma$=0.8，利用算法4.1计算，在策略$\pi$下，确定环境扫地机器人任务的状态值函数。
<center><img src='./第四章图片/图4.2.png' width='200'></center>
<center>图4.1 扫地机器人环境</center>

计算过程如下：<br>
（1）当$\iota=0$时，对所有的$s$初始化为：$V(s)=0$<br>
（2）当$\iota=1$时，以状态$S_{5}$、$S_{17}$、$S_{24}$为例：
<center>
    <img src='./第四章图片/图4.3.png' width='600'>
    <img src='./第四章图片/图4.4.png' width='600'>
    <img src='./第四章图片/图4.5.png' width='600'></center><br>
按顺序计算完一轮后，得到值函数$V(s)$<br>
（3）当$\iota=2$时，以状态$S_{5}$、$S_{17}$、$S_{24}$为例。采用<b>异步</b>计算方式：
<center><img src='./第四章图片/图4.6.png' width='600'></center><br>
（4）当$\iota=30$时，$|V_\iota(s)-V_{\iota-1}(s)|<\Delta$，认为$V_\iota(s)$已经收敛于$v_\pi(s)$，计算得到的$v_\pi(s)$就是在策略$\pi$下的有效评估

随着迭代的进行，每轮状态值函数的更新过程如下：
<center><img src='./第四章图片/图4.7.png'></center>

In [1]:
import numpy as np


def reward(s):

    if s == 20:  # 到充电站
        return 1
    elif s == 12:  # 到陷阱中
        return -10
    elif s == 9:  # 到垃圾处
        return 3
    else:
        return 0  # 其他
    # in表示0是[*，*，*]中的一个


def getAction(a):
    if a == 'n':
        return 0
    elif a == 'e':
        return 1
    elif a =='s':
        return 2
    elif a == 'w':
        return 3

# 在s状态下执行动作a，返回下一状态（编号）
def next_states(s, a):
    # 越过边界时pass
    if (s < world_w and a == 'n') \
            or (s % world_w == 0 and a == 'w') \
            or (s > length - world_w - 1 and a == 's') \
            or ((s + 1) % world_w == 0 and a == 'e'):  # (s % (world_w - 1) == 0 and a == 'e' and s != 0)
        next_state = s  # 表现为next_state不变
    else:
        next_state = s + ds_action[a]  # 进入下一个状态
    return next_state


# 在s状态下执行动作，返回所有可能的下一状态（编号）list
def getsuccessor(s):
    successor = []
    for a in action:  # 遍历四个动作
        if s == next_states(s,a):
            continue
        else:
            # print("状态s=%s,动作a=%s"%(s,a))
            next = next_states(s, a)  # 得到下一个状态（编号）
        successor.append(next)  # 以list保存当前状态s下执行四个动作的下一状态
    # print(len(successor))
    return successor

def initPolicy():
    for s in range(length):
        for a in action:
            if next_states(s,a) == s:
                continue
            newAction = getAction(a)
            policy[s][newAction] = 1 / len(getsuccessor(s))
    # print(policy)


def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    iter = 0

    while True:
        k = -1
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]:  # 遍历所有状态 [0~25]
            k += 1
            if s in [9, 20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            print("第%d 的状态"%(k),end="")
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s,a)
                rewards = reward(next_state)
                if next_state == 12:
                    v += policy[s][newAction] * (rewards + gamma * V[s])
                    # print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
                else:
                    v += policy[s][newAction] * (rewards + gamma * V[next_state])
            #     print("%.2f*(%d+%.1f*%.2f)+" % (policy[s][newAction], rewards, gamma, value[next_state]), end="")
            # print()
            # successor = getsuccessor(s)
            # for next_state in successor:
            #     rewards = reward(next_state)
            #     v += 1 / len(successor) * (rewards + gamma * V[next_state])
                    print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
            print("v = %.3f"%(v))

            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            V[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
        value = np.array(V).reshape(world_h, world_w)
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(np.round(value, decimals=4))
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
    return V  # 一轮迭代结束后，状态值函数暂时固定


def Caculate_Q(s, V, discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    for a in action:  # 遍历能走的动作
        # for prob, next_state, reward, done in env.P[s][a]:
        #     Q[s][a] += prob * (reward + discount_factor * V[next_state])  # 计算当前状态s下的动作值函数列表 [q1,q2,q3,q4]
        next_state = next_states(s,a)
        if next_state == s: #碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        Q[s][numberA]= rewards + discount_factor * V[next_state]
        print("Q[%s][%s]  %.2f = %s + 0.8 * %.2f:"%(s,a,Q[s][numberA],rewards,V[next_state]))
    return Q


world_h = 5
world_w = 5
length = world_h * world_w
gamma = 0.8
state = [i for i in range(length)]  # 状态（编号）
action = ['n', 'e', 's', 'w']  # 动作名称
ds_action = {'n': -world_w, 'e': 1, 's': world_w, 'w': -1}
value = [0 for i in range(length)]  # 初始化状态值函数，均为0.  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
policy = np.zeros([length, len(action)])


def main():
    initPolicy()
    value = policy_eval()
    # v = np.array(value).reshape(world_h, world_w)
    # print(np.round(v, decimals=2))
    #
    # print(Caculate_Q(3,value))
    # initPolicy()
if __name__ == '__main__':
    main()
# policy, V = Policy_Iterration()

第1 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(1+0.8*0.000)+v = 0.333
第2 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.333)+v = 0.089
第3 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.089)+v = 0.024
第4 的状态 0.50*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.50*(0+0.8*0.024)+v = 0.009
第5 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+ 0.33*(1+0.8*0.000)+ 0.00*(0+0.8*0.000)+v = 0.333
第6 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.333)+ 0.25*(0+0.8*0.333)+v = 0.133
第7 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.089)+ 0.25*(0+0.8*0.133)+v = -2.456
第8 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.024)+ 0.25*(0+0.8*-2.456)+v = -0.486
第9 的状态 0.33*(0+0.8*0.000)+ 0.00*(0+0.8*0.000)+ 0.33*(0+0.8*0.009)+ 0.33*(0+0.8*-0.486)+v = -0.127
第10 的状态 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.000)+ 0.33*(0+0.8*0.333)+ 0.00*(0+0.8*0.000)+v = 0.089
第11 的状态 0.25*(0+0.8*0.000)+ 0.25*(0+0.8*0.133)+ 0.25*(

第2 的状态 0.33*(0+0.8*-4.649)+ 0.33*(0+0.8*-1.280)+ 0.00*(0+0.8*-1.772)+ 0.33*(0+0.8*-0.716)+v = -1.772
第3 的状态 0.33*(0+0.8*-2.160)+ 0.33*(0+0.8*-0.867)+ 0.00*(0+0.8*-1.280)+ 0.33*(0+0.8*-1.772)+v = -1.280
第4 的状态 0.50*(0+0.8*-0.887)+ 0.00*(0+0.8*-0.867)+ 0.00*(0+0.8*-0.867)+ 0.50*(0+0.8*-1.280)+v = -0.867
第5 的状态 0.33*(0+0.8*-1.831)+ 0.33*(0+0.8*-2.162)+ 0.33*(1+0.8*0.000)+ 0.00*(0+0.8*-0.731)+v = -0.731
第6 的状态 0.25*(0+0.8*-4.716)+ 0.25*(0+0.8*-4.649)+ 0.25*(0+0.8*-0.716)+ 0.25*(0+0.8*-0.731)+v = -2.162
第7 的状态 0.25*(0+0.8*-2.160)+ 0.25*(0+0.8*-1.772)+ 0.25*(0+0.8*-2.162)+v = -4.649
第8 的状态 0.25*(0+0.8*-3.987)+ 0.25*(0+0.8*-0.887)+ 0.25*(0+0.8*-1.280)+ 0.25*(0+0.8*-4.649)+v = -2.160
第9 的状态 0.33*(0+0.8*-0.300)+ 0.00*(0+0.8*-0.887)+ 0.33*(0+0.8*-0.867)+ 0.33*(0+0.8*-2.160)+v = -0.887
第10 的状态 0.33*(0+0.8*-1.417)+ 0.33*(0+0.8*-4.716)+ 0.33*(0+0.8*-0.731)+ 0.00*(0+0.8*-1.831)+v = -1.831
第11 的状态 0.25*(0+0.8*-2.372)+ 0.25*(0+0.8*-2.162)+ 0.25*(0+0.8*-1.831)+v = -4.716
第13 的状态 0.25*(0+0.8*-0.987)+ 0.

**例4.2**：用同步迭代解决上述问题

In [1]:
import numpy as np


# 定义奖励
def reward(s):
    if s == 20:  # 到充电站
        return 1
    elif s == 12:  # 到陷阱中
        return -10
    elif s == 9:  # 到垃圾处
        return 3
    else:
        return 0  # 其他
    # in表示0是[*，*，*]中的一个


def getAction(a):
    if a == 'n':
        return 0
    elif a == 'e':
        return 1
    elif a == 's':
        return 2
    elif a == 'w':
        return 3


# 在s状态下执行动作a，返回下一状态（编号）
def next_states(s, a):
    # 越过边界时pass
    if (s < world_w and a == 'n') \
            or (s % world_w == 0 and a == 'w') \
            or (s > length - world_w - 1 and a == 's') \
            or ((s + 1) % world_w == 0 and a == 'e'):  # (s % (world_w - 1) == 0 and a == 'e' and s != 0)
        next_state = s  # 表现为next_state不变
    else:
        next_state = s + ds_action[a]  # 进入下一个状态
    return next_state


# 在s状态下执行动作，返回所有可能的下一状态（编号）list
def getsuccessor(s):
    successor = []
    for a in action:  # 遍历四个动作
        if s == next_states(s, a):
            continue
        else:
            next = next_states(s, a)  # 得到下一个状态（编号）
        successor.append(next)  # 以list保存当前状态s下执行四个动作的下一状态
    return successor


def initPolicy():
    for s in range(length):
        for a in action:
            if next_states(s, a) == s:
                continue
            newAction = getAction(a)
            policy[s][newAction] = 1 / len(getsuccessor(s))

def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    Value = np.zeros(length)
    iter = 0
    while True:
        delta = 0  # 定义最大差值，判断是否有进行更新
        for s in [20, 21, 22, 23, 24, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4]:  # 遍历所有状态 [0~25]
            if s in [9, 20, 12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s, a)
                rewards = reward(next_state)
                if next_state == 12:
                    v += policy[s][newAction] * (rewards + gamma * V[s])
                    # print(" %.2f*(%d+%.1f*%.3f)+" % (policy[s][newAction], rewards, gamma, V[next_state]), end="")
                else:
                    v += policy[s][newAction] * (rewards + gamma * V[next_state])
            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            Value[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)
        value = np.array(Value).reshape(world_h, world_w)
        iter += 1
        print('k=', iter)  # 打印迭代次数
        print(np.round(value, decimals=4))
        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
        V = Value.copy()
    return V  # 一轮迭代结束后，状态值函数暂时固定


def Caculate_Q(s, V, discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    for a in action:  # 遍历能走的动作
        next_state = next_states(s, a)
        if next_state == s:  # 碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        Q[s][numberA] = rewards + discount_factor * V[next_state]
        print("Q[%s][%s]  %.2f = %s + 0.8 * %.2f:" % (s, a, Q[s][numberA], rewards, V[next_state]))
    return Q


world_h = 5
world_w = 5
length = world_h * world_w
gamma = 0.8
state = [i for i in range(length)]  # 状态（编号）
action = ['n', 'e', 's', 'w']  # 动作名称
ds_action = {'n': -world_w, 'e': 1, 's': world_w, 'w': -1}
value = [0 for i in range(length)]  # 初始化状态值函数，均为0.  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
policy = np.zeros([length, len(action)])


def main():
    initPolicy()
    value = policy_eval()

if __name__ == '__main__':
    main()

k= 1
[[ 0.      0.      0.      0.      1.5   ]
 [ 0.      0.     -2.5     0.75    0.    ]
 [ 0.     -2.5     0.     -2.5     1.    ]
 [ 0.3333  0.     -2.5     0.      0.    ]
 [ 0.      0.3333  0.      0.      0.    ]]
k= 2
[[ 0.      0.     -0.6667  0.6     1.5   ]
 [ 0.     -1.     -2.85   -0.25    0.    ]
 [-0.5778 -3.      0.     -2.65    0.3333]
 [ 0.3333 -0.8667 -3.     -1.      0.2667]
 [ 0.      0.3333 -0.5778  0.      0.    ]]
k= 3
[[ 0.     -0.4444 -0.6     0.1556  1.74  ]
 [-0.4207 -1.17   -3.4533 -0.23    0.    ]
 [-0.7111 -3.5889  0.     -3.2133  0.3644]
 [-0.0519 -1.0667 -3.5889 -1.0767 -0.1778]
 [ 0.     -0.0519 -0.7111 -0.4207  0.1067]]
k= 4
[[-0.3461 -0.472  -0.9979  0.2427  1.5622]
 [-0.5016 -1.5815 -3.5907 -0.5522  0.    ]
 [-1.0831 -3.8073  0.     -3.3311  0.0957]
 [-0.1407 -1.4563 -3.7887 -1.4801 -0.1615]
 [ 0.     -0.1407 -1.0831 -0.4483 -0.2394]]
k= 5
[[-3.8950e-01 -7.8010e-01 -1.0187e+00  3.2000e-03  1.5971e+00]
 [-8.0280e-01 -1.6743e+00 -3.8445e+00 -5.8580e-0

## 4.2 基于状态值函数的策略迭代

&emsp;&emsp;基于状态值函数的策略迭代算法主要包括以下3个阶段：<br>
（1）初始化策略函数$\pi(a|s)$和状态值函数$V_0(s)$<br>
（2）策略评估：在当前策略$\pi$下，使用贝尔曼方程更新状态值函数$V_\iota(s)$，直到收敛于$v_\pi(s)$，在根据公式$q_\pi(s,a)=\sum p(s',r|s,a)[r+\Delta v_\pi(s')]$计算出$q_\pi(s,a)$<br>
（3）策略改进：基于$q_\pi(s,a)$，通过贪心策略得到更优策略$\pi'$。<br>
<center><img src='./第四章图片/图4.8.png'></center>

**例4.3** 将基于状态值函数的策略迭代算法4.3应用于例4.1的确定环境扫地机器人任务中，经过多轮迭代后，得到如下表所示的值函数和策略迭代更新过程。
<center><img src='./第四章图片/图4.9.png' width='700'></center>
<center>面向确定环境扫地机器人任务的状态值函数策略迭代更新过程</center>
可以得到如下状态值函数及策略迭代更新图：
<center><img src='./第四章图片/图4.10.png' width='500'></center>
<center>面向确定环境扫地机器人任务的状态值函数及策略迭代更新图</center>
$emsp;$emsp;在上图中，左边一列给出了每一轮（l=0,1,2,3,4,5）状态值函数的迭代更新过程。当l=4和5时，策略不再发生变化（这里状态值也不再变化），策略评估迭代收敛。右侧一列给出了相应的策略改进过程。一句对动作值函数的贪心，获得较好策略，对策略进行改进。由上图可以看出，每个状态的最优策略始终选择动作值最大的动作。

In [2]:
import numpy as np


def reward(s):
    if s == 20:  # 到充电站
        return 1
    elif s == 12:  # 到陷阱中
        return -10
    elif s == 9:  # 到垃圾处
        return 3
    else:
        return 0  # 其他


def getAction(a):
    if a == 'n':
        return 0
    elif a == 'e':
        return 3
    elif a == 's':
        return 1
    elif a == 'w':
        return 2

# 在s状态下执行动作a，返回下一状态（编号）
def next_states(s, a):
    # 越过边界时pass
    if (s < world_w and a == 'n') \
            or (s % world_w == 0 and a == 'w') \
            or (s > length - world_w - 1 and a == 's') \
            or ((s + 1) % world_w == 0 and a == 'e'):  # (s % (world_w - 1) == 0 and a == 'e' and s != 0)
        next_state = s  # 表现为next_state不变
    else:
        next_state = s + ds_action[a]  # 进入下一个状态
    return next_state


# 在s状态下执行动作，返回所有可能的下一状态（编号）list
def getsuccessor(s):
    successor = []
    for a in action:  # 遍历四个动作
        if s == next_states(s, a):
            continue
        else:
            # print("状态s=%s,动作a=%s"%(s,a))
            next = next_states(s, a)  # 得到下一个状态（编号）
        successor.append(next)  # 以list保存当前状态s下执行四个动作的下一状态
    return successor


def initPolicy():
    for s in range(length):
        if s == 9 or s == 12 or s == 20:
            continue
        for a in action:
            if next_states(s, a) == s:
                continue
            newAction = getAction(a)
            policy[s][newAction] = 1 / len(getsuccessor(s))


def policy_eval(theta=0.000001):
    V = np.zeros(length)  # 初始化状态值函数列表
    while True:
        delta = 0  # 定义最大差值，判断是否有进行更新

        for s in [20, 21, 22, 23, 24, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4]:  # 遍历所有状态 [0~25]
            if s in [9, 20, 12]:  # 若当前状态为吸入状态，则直接pass不做操作
                continue
            v = 0  # 针对每个状态值函数进行计算
            for a in action:
                newAction = getAction(a)
                next_state = next_states(s, a)
                rewards = reward(next_state)
                if next_state == 12:
                    v += policy[s][newAction] * (rewards + gamma * V[s])
                else:
                    v += policy[s][newAction] * (rewards + gamma * V[next_state])
            delta = max(delta, np.abs(v - V[s]))  # 更新差值
            V[s] = v  # 存储(更新)每个状态下的状态值函数，即伪代码中的 v <- V(s)

        if delta < theta:  # 策略评估的迭代次数不能太多，否则状态值函数的数值会越来越大（即使算法仍然在收敛）
            break
    return V  # 一轮迭代结束后，状态值函数暂时固定


def Caculate_Q(s, V, num, discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    Q[:][:] = -100
    for a in action:  # 遍历能走的动作
        next_state = next_states(s, a)
        if next_state == s:  # 碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        if s == 12:
            Q[s][numberA] = rewards + discount_factor * V[s]
        else:
            Q[s][numberA] = rewards + discount_factor * V[next_state]
        print("Q[%s][%d]  %f = %s + 0.8 * %.2f:" % (num, numberA, Q[s][numberA], rewards, V[next_state]))
    return Q


def policy_improvement(V, policy):  # 策略改进
    """
    Returns:
        policy: the optimal policy, a matrix of shape [S, A] where each state s contains a valid probability distribution over actions.
        V: the value function for the optimal policy.
    """
    k = -1
    policy_stable = True  # Will be set to false if we make any changes to the policy
    for s in [20, 21, 22, 23, 24, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4]:
        k += 1
        if np.all(policy[s] == 0):
            continue
        old_a = np.argmax(policy[s])  # 记录当前策略在该状态s下所选择的动作 —— 选择概率最高的动作
        Q = Caculate_Q(s, V, k)  # 在当前状态和策略下，计算动作值函数 —— 要判断在状态s下选择其他动作的好坏，就需要获得状态s的动作值函数
        best_a = np.argmax(Q[s])  # 采用贪婪策略获得状态s的最优动作；如果往两个方向前进都可以得到最优解，会随机选其一
        if old_a != best_a:  # 判定策略是否稳定
            policy_stable = False  # 动作还在变化，不稳定状态
        policy[s] = np.eye(len(action))[best_a]  # 基于贪婪法则更新策略，形如[0，0，1，0]； -》np.eye(*)：构建对角单位阵
        # 经过一次策略改进后的策略将不再拥有多个动作可供备选，取而代之的是在某种状态下的确定策略
    return V, policy_stable


world_h = 5
world_w = 5
length = world_h * world_w
gamma = 0.8
state = [i for i in range(length)]  # 状态（编号）
action = ['n', 's', 'w', 'e']  # 动作名称
ds_action = {'n': -world_w, 'e': 1, 's': world_w, 'w': -1}
value = [0 for i in range(length)]  # 初始化状态值函数，均为0.  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
policy = np.zeros([length, len(action)])

def Policy_Iterration():
    initPolicy()
    k = 1
    while True:  # Evaluate the current policy
        print("迭代次数", k)
        V = policy_eval()  # 得到当前策略下的收敛状态值函数 —— 与Value_Iteration的不同点，多了policy_eval()函数。policy会在迭代中改变
        V, policy_stable = policy_improvement(V, policy)
        v = np.array(V).reshape(world_h, world_w)
        policy1 = np.array(policy).reshape(length, len(action))
        print(np.round(v, decimals=4))
        print("*" * 100)
        k += 1
        if policy_stable:  # # If the policy is stable we've found an optimal policy. Return it
            return policy, V

policy, V = Policy_Iterration()


迭代次数 1
Q[1][0]  -1.729965 = 0 + 0.8 * -2.16:
Q[1][2]  1.000000 = 1 + 0.8 * 0.00:
Q[1][3]  -1.417435 = 0 + 0.8 * -1.77:
Q[2][0]  -3.718945 = 0 + 0.8 * -4.65:
Q[2][2]  -0.572640 = 0 + 0.8 * -0.72:
Q[2][3]  -1.023796 = 0 + 0.8 * -1.28:
Q[3][0]  -1.728382 = 0 + 0.8 * -2.16:
Q[3][2]  -1.417435 = 0 + 0.8 * -1.77:
Q[3][3]  -0.693420 = 0 + 0.8 * -0.87:
Q[4][0]  -0.709755 = 0 + 0.8 * -0.89:
Q[4][2]  -1.023796 = 0 + 0.8 * -1.28:
Q[5][0]  -1.464471 = 0 + 0.8 * -1.83:
Q[5][1]  1.000000 = 1 + 0.8 * 0.00:
Q[5][3]  -1.729965 = 0 + 0.8 * -2.16:
Q[6][0]  -3.773060 = 0 + 0.8 * -4.72:
Q[6][1]  -0.572640 = 0 + 0.8 * -0.72:
Q[6][2]  -0.585183 = 0 + 0.8 * -0.73:
Q[6][3]  -3.718945 = 0 + 0.8 * -4.65:
Q[7][0]  -10.000000 = -10 + 0.8 * 0.00:
Q[7][1]  -1.417435 = 0 + 0.8 * -1.77:
Q[7][2]  -1.729965 = 0 + 0.8 * -2.16:
Q[7][3]  -1.728382 = 0 + 0.8 * -2.16:
Q[8][0]  -3.189413 = 0 + 0.8 * -3.99:
Q[8][1]  -1.023796 = 0 + 0.8 * -1.28:
Q[8][2]  -3.718945 = 0 + 0.8 * -4.65:
Q[8][3]  -0.709755 = 0 + 0.8 * -0.89:
Q[9][0]

## 4.3 值迭代

&emsp;&emsp;在策略迭代中，每轮策略改进之前都涉及策略评估，每次策略评估都需要多次遍历才能保证状态值函数在一定程度上得到收敛，这将消耗大量的时间和计算资源。从例 4.1 中可以看出，当$\tau\geq 4$时，值函数的变化对策略不再有影响。于是根据迭代次数与策略稳定的相互关系，考虑在单步评估之后就进入改进过程，即采取截断式策略评估，在一次遍历完所有的状态后立即停止策略评估，进行策略改进，这种方法称为值迭代。基于状态值函数的值迭代公式为：

\begin{align}
v_\iota(s) &= \max \limits_a \mathop{\mathbb{E}}_\pi(R_{t+1}+\gamma v_\pi(S_{t+1})|S_t=s,A_t=a) \\
        &=\max \limits_a \sum_{s^{'},r}p(s^{'},r|s,a)[r+\gamma v_{\iota -1}(s^{'})]   
\end{align}

算法4.5给出了在有穷状态空间MDP中，基于状态值函数的值迭代算法。

<center><img src='./第四章图片/图4.11.png' width='600'></center>
&emsp;&emsp;与策略迭代类似，也可以直接基于动作值函数进行值迭代，迭代公式为：
$$ \mathcal{Q}(s,a)=\sum_{s^{'},r}p(s^{'},r|s,a)\left [r+\gamma\max_{a^{'}} \mathcal{Q}(s^{'},a^{'})\right]$$

算法4.6给出了在有穷状态空间MDP中，基于动作值函数的值迭代算法。
<center><img src='./第四章图片/图4.12.png' width='600'></center>
<center><img src='./第四章图片/图4.13.png' width='600'></center>
&emsp;&emsp;算法4.5和算法4.6既可用于确定环境MDP又可用于随机环境MDP。<br>

**例4.4**&ensp;将基于状态值函数的值迭代算法4.5应用于例4.1的确定环境扫地机器人任务中，可以得到以下状态值迭代更新过程：<br>
（1）当$l=0$时（$l$为值迭代的次数）,对于所有的$s$初始化为：$V(s)=0$;<br>
（2）当$l=1$时,以状态$S_{24}$为例。在策略$\pi$下，只能采取向下和向左2个动作，概率各为0.5。采取向下的动作时，到达状态$S_{19}(V(S_{23})=0)$，并可以捡到垃圾，获得$r_1=+3$的奖赏；采取向左的动作时，到达状态$S_{23}(V(S_{23})=2.40)$，获得$r_2=0$的奖赏。根据算法4.2计算得：

\begin{align}
V(s_{24}) &= \max (r_1 + \gamma V(S_{19}),r_2+\gamma V(S_{23})) \\
        &=\max (3+0.8*0,0+0.8*2.40)\\
        &=\max (3,1.92) \\
        &= 3.00
\end{align}

同理可以计算状态S_5和S_6的状态值函数：
\begin{align}
V(s_{6}) &= \max (r_1 + \gamma V(S_{10}),r_2+\gamma V(S_{0}),r_3+\gamma V(S_{6})) \\
        &=\max (0+0.8*0,1+0.8*0,0+0.8*{0})\\
        &= 3.00
\end{align}

\begin{align}
V(s_{6}) &= \max (r_1 + \gamma V(S_{11}),r_2+\gamma V(S_{1}),r_3+\gamma V(S_{5}),r_4+\gamma V(S_{7})) \\
        &=\max (0+0.8*0,0+0.8*1.00,0+0.8*{1.00},0+0.8*{0})\\
        &= 0.80
\end{align}

按顺序计算完一轮后，得到值函数$V(s)$，如图4.7$(l=1)$所示。<br>

（3）当$l=2$时,以状态$S_{22}$、$S_{23}$为例，计算状态值函数。异步计算方式，通常与迭代的计算顺序有关，根据例4.1规定，在每一轮次中，这3个状态的计算顺序为：

\begin{align}
V(s_{22})&= \max (r_1 + \gamma V(S_{17}),r_2+\gamma V(S_{21}),r_3+\gamma V(S_{23})) \\
        &=\max (0+0.8*{2.40},0+0.8*{0.41},0+0.8*{2.40})\\
        &= 1.92
\end{align}
\begin{align}
V(s_{23})&= \max (r_1 + \gamma V(S_{18}),r_2+\gamma V(S_{22}),r_3+\gamma V(S_{24})) \\
        &=\max (0+0.8*{3.00},0+0.8*{1.92},0+0.8*{3.00})\\
        &= 3.00
\end{align}

按顺序计算完一轮后，得到值函数$V(s)$，如图 4.7（ $l=2$）所示。<br>
（4）当$l=6$时，$|v_{l}(s)-v_{l-1}(s)|_{\infty}<\theta$，认为v_{l}(s)已经收敛于$v_*(s)$,计算得到的$v_*(s)$就是最优状态值函数。<br>

确定环境扫地机器人任务通过状态值迭代得到的结果如下图所示：
<center><img src='./第四章图片/图4.14.png' width='600'></center>

In [4]:
import numpy as np

# 定义奖励
def reward(s):

    if s == 20:  # 到充电站
        return 1
    elif s == 12:  # 到陷阱中
        return -10
    elif s == 9:  # 到垃圾处
        return 3
    else:
        return 0  # 其他
    # in表示0是[*，*，*]中的一个

def getAction(a):
    if a == 'n':
        return 2
    elif a == 'e':
        return 1
    elif a =='s':
        return 3
    elif a == 'w':
        return 0

# 在s状态下执行动作a，返回下一状态（编号）
def next_states(s, a):
    # 越过边界时pass
    if (s < world_w and a == 'n') \
            or (s % world_w == 0 and a == 'w') \
            or (s > length - world_w - 1 and a == 's') \
            or ((s + 1) % world_w == 0 and a == 'e'):  # (s % (world_w - 1) == 0 and a == 'e' and s != 0)
        next_state = s  # 表现为next_state不变
    else:
        next_state = s + ds_action[a]  # 进入下一个状态
    return next_state


# 在s状态下执行动作，返回所有可能的下一状态（编号）list
def getsuccessor(s):
    successor = []
    for a in action:  # 遍历四个动作
        if s == next_states(s,a):
            continue
        else:
            # print("状态s=%s,动作a=%s"%(s,a))
            next = next_states(s, a)  # 得到下一个状态（编号）
        successor.append(next)  # 以list保存当前状态s下执行四个动作的下一状态
    # print(len(successor))
    return successor


# 更新状态值函数
def value_update(s,numb):  # 传入当前状态
    value_new = 0
    if s in [9,20,12]:  # 若当前状态为吸入状态，则直接pass不做操作
        pass
    else:
        Q =[]
        successor = getsuccessor(s)  # 得到所有可能的下一状态list
        # rewards = reward(s)  # 得到当前状态的奖励
        # print("s %s="%s,end="")
        for next_state in successor:  # 遍历所有可能的下一状态
            rewards = reward(next_state)
            value_new = rewards + gamma * value[next_state]  # 计算公式，得到当前状态的状态价值函数
            Q.append(value_new)
            # print("%.2f*(%d+%.1f*%.2f) = %.2f"%(1/len(successor),rewards,gamma,value[next_state],value_new))
            # 注意前面的1/len(successor)为该s状态能够到下个状态的个数概率，该代码是第一次迭代时的固定策略π(a|s)
        value_new = max(Q)
        # print("第%d个状态最大值：%.2f"%(numb,value_new))
        # print()
    return value_new


def Caculate_Q(s, V, num,discount_factor=0.8):
    """
    Returns:
        A vector of length env.nA containing the expected value of each action.
    """
    Q = np.zeros((length, 4))
    # Q[:][:] = -1000
    for a in ['w', 'e', 'n', 's']:  # 遍历能走的动作
        # for prob, next_state, reward, done in env.P[s][a]:
        #     Q[s][a] += prob * (reward + discount_factor * V[next_state])  # 计算当前状态s下的动作值函数列表 [q1,q2,q3,q4]
        next_state = next_states(s,a)
        if next_state == s: #碰壁
            continue
        rewards = reward(next_state)
        numberA = getAction(a)
        Q[s][numberA]= rewards + discount_factor * V[next_state]
        print("Q[%s][%d]  %.2f = %s + 0.8 * %.2f:"%(num,numberA,Q[s][numberA],rewards,V[next_state]))


def initial_state():
    v = np.array(value).reshape(world_h, world_w)  # 调整初始化状态值函数矩阵
    print(v)


world_h = 5
world_w = 5
length = world_h * world_w
gamma = 0.8
state = [i for i in range(length)]  # 状态（编号）
action = ['n', 'e', 's', 'w']  # 动作名称
ds_action = {'n': -world_w, 'e': 1, 's': world_w, 'w': -1}
value = [0 for i in range(length)]  # 初始化状态值函数，均为0.  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


def main():
    max_iter = 7  # 最大迭代次数
    initial_state()

    iter = 1

    while iter < max_iter:
        numb = -1
        for s in [20,21,22,23,24,15,16,17,18,19,10,11,12,13,14,5,6,7,8,9,0,1,2,3,4]:  # 遍历所有状态
            numb += 1
            value[s] = value_update(s,numb)  # 更新状态值函数
            Caculate_Q(s,value,numb)
        v = np.array(value).reshape(world_h, world_w)  # 更新状态值函数矩阵

        if (iter <= 10) or (iter % 10 == 0):  # 前1次 + 每10次打印一次
            print('k=', iter)  # 打印迭代次数
            print(np.round(v, decimals=4))  # np.round() 返回浮点数的四舍五入值

        iter += 1


if __name__ == '__main__':
    main()


[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Q[0][1]  0.00 = 0 + 0.8 * 0.00:
Q[0][2]  0.00 = 0 + 0.8 * 0.00:
Q[1][0]  1.00 = 1 + 0.8 * 0.00:
Q[1][1]  0.00 = 0 + 0.8 * 0.00:
Q[1][2]  0.00 = 0 + 0.8 * 0.00:
Q[2][0]  0.80 = 0 + 0.8 * 1.00:
Q[2][1]  0.00 = 0 + 0.8 * 0.00:
Q[2][2]  0.00 = 0 + 0.8 * 0.00:
Q[3][0]  0.64 = 0 + 0.8 * 0.80:
Q[3][1]  0.00 = 0 + 0.8 * 0.00:
Q[3][2]  0.00 = 0 + 0.8 * 0.00:
Q[4][0]  0.51 = 0 + 0.8 * 0.64:
Q[4][2]  0.00 = 0 + 0.8 * 0.00:
Q[5][1]  0.00 = 0 + 0.8 * 0.00:
Q[5][2]  0.00 = 0 + 0.8 * 0.00:
Q[5][3]  1.00 = 1 + 0.8 * 0.00:
Q[6][0]  0.80 = 0 + 0.8 * 1.00:
Q[6][1]  0.00 = 0 + 0.8 * 0.00:
Q[6][2]  0.00 = 0 + 0.8 * 0.00:
Q[6][3]  0.80 = 0 + 0.8 * 1.00:
Q[7][0]  0.64 = 0 + 0.8 * 0.80:
Q[7][1]  0.00 = 0 + 0.8 * 0.00:
Q[7][2]  -10.00 = -10 + 0.8 * 0.00:
Q[7][3]  0.64 = 0 + 0.8 * 0.80:
Q[8][0]  0.51 = 0 + 0.8 * 0.64:
Q[8][1]  0.00 = 0 + 0.8 * 0.00:
Q[8][2]  0.00 = 0 + 0.8 * 0.00:
Q[8][3]  0.51 = 0 + 0.8 * 0.64:
Q[9][0]  0.41 = 0 + 0.8 * 0.51:
Q[

### 汽车租赁问题

&emsp;&emsp;汽车租赁问题。杰克经营两家异地的汽车租赁场（A，B 租赁场），每天都会有客户来租车。如果每个租赁场有可供外租的汽车，每租出一辆汽车，杰克都会获得10美元的租金。为了保证每个租赁场有足够的车辆可租，每天晚上杰克会在两个租赁场之间移动车辆，每辆车的移车费用为2美元。假设每个租赁场的租车和还车的数量是一个泊松随机量，即期望数量n的概率为$\frac{\lambda_{n}}{n!}e^{-\lambda}$，其中$\lambda$分别是$\lambda_{A1}=3$，$\lambda_{B1}=4$，还车的$\lambda$分别是$\lambda_{A2}=3$，$\lambda_{B2}=2$。<br>
&emsp;&emsp;为了简化问题，给定3个假设：（1）任何一个租赁场车辆总数不超过20辆车；（2）当天还回的车辆第2天才能出租。（3）两个租车场之间每天最多可移车数量为5辆。<br>
&emsp;&emsp;利用策略迭代方法，在折扣率$\gamma = 0.9$时，计算两个租赁场点之间的最优移车策略。<br>
&emsp;&emsp;将汽车租赁问题描述为一个持续的有限 MDP 模型。其中时刻 t 按天计算；状态为每天租还车结束时，两个租赁场的车辆数；动作为每天晚上在两个租赁场之间移动的车辆数目。<br>
&emsp;&emsp;该问题的 MDP 模型为：<br>
&emsp;&emsp;（1）状态空间：两个租赁场每天租还结束时，可供出租的车辆数组成的二维向量，状态共 $21*21=441$个，即$\mathcal{S}={[0,0],[0,1],[0,2],...,[10,10],[10,11],....,[20,19][20,20]}$。用$s$表示状态空间中的某一个状态$[s_A,s_B]$。例如，$[3,5]$表示每天租还结束后，A 租赁场可供出租 3 辆车、B 租赁场可供出租 5 辆车的状态。<br>
&emsp;&emsp;（2）动作空间：每天晚上在两个租赁场之间移动车辆的数目。根据任务的假设，可移动车辆数目不超过 5 辆。设 A 租赁场向 B 租赁场移车为“-”，B 租赁场向 A 租赁场移车为“ + ” 。 该 问 题 的 动 作 空 间 中 共 包 含 11 个 动 作 ， 即 ：$\mathcal{A}(s)={-5,-4,-3,-2,-1,0,+1,+2,+3,+4,+5}$。用$a$表示动作空间中的某一个动作。例如，-2 表示 A 租赁场向 B 租赁场移车 2 辆；+3 表示 B 租赁场向 A 租赁场移车3辆。<br>
&emsp;&emsp;（3）状态转移函数：
&emsp;&emsp;令$s=[s_A,s_B]$，$s^{'}=s^{'}_A,s^{'}_B$，那么状态转移函数为：$p([s_A,s_B]),a,[s^{'}_A,s^{'}_B]$，即在当前状态$s=[s_A,s_B]$下，采取动作a，到达下一状态$s^{'}=s^{'}_A,s^{'}_B$的概率。设$n_A$为A租赁场当天租掉的车辆数；$n_B$为B租赁场当天租掉的车辆数。 $m_A$为A租赁场当天还回的车辆数；$m_B$为B租赁场当天还回的车辆数。假设当前状态为$s=[2,5]$，采取+1动作后，中间状态为$s_L=[3,4]$，当下一状态为$s^{'}=[1,2]$，对于 A 租赁场来说，一天的租掉、还回车辆$(n_A,m_A)$可能为：$(2,0)、(3,1)$共2种情况；对于B租赁场来说，一天的租掉、换回车辆$(n_B,m_B)$可能为：$(2,0)、(3,1)、(4,2)$共3种情况。那么在该情况下，状态转移函数$p([2,5],+1,[1,2])$计算为：
\begin{align}
p([2,5],+1,[1,2]) &= \left( \frac{\lambda_{A1}^2}{2!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^0}{0!}e^{-\lambda ^{A1}}\right)
* \left( \frac{\lambda_{B1}^2}{2!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^0}{0!}e^{-\lambda ^{B1}}\right) \\
 & + \left( \frac{\lambda_{A1}^2}{2!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^0}{0!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{B1}^3}{3!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^1}{1!}e^{-\lambda ^{B1}}\right) \\
 & + \left( \frac{\lambda_{A1}^2}{2!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^0}{0!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{B1}^4}{4!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^2}{2!}e^{-\lambda ^{B1}}\right) \\
 & + \left( \frac{\lambda_{A1}^3}{3!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^1}{1!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{B1}^2}{2!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^0}{0!}e^{-\lambda ^{B1}}\right) \\
 & + \left( \frac{\lambda_{A1}^3}{3!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^1}{1!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{B1}^3}{3!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^1}{1!}e^{-\lambda ^{B1}}\right) \\
 & + \left( \frac{\lambda_{A1}^3}{3!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{A1}^1}{1!}e^{-\lambda ^{A1}}\right)* \left( \frac{\lambda_{B1}^4}{4!}e^{-\lambda ^{B1}}\right)* \left( \frac{\lambda_{B1}^2}{2!}e^{-\lambda ^{B1}}\right)
\end{align}
根据任务可知，这里$\lambda _{A1}=3,\lambda _{B1}=4,\lambda _{A2}=3,\lambda _{B2}=2$。<br>
综上所述，如果$0\le S^{'}_A-S_A-a+n_A\le 20$且$0\le S^{'}_B-S_B-a+n_B\le 20$，则有：<br><br>
\begin{align}
p(s,a,s^{'})&= p([s_A,s_B],a,[s^{'}=s^{'}_A,s^{'}_B]) \\
        &= \sum _{n_A=0}^{s_A+a} \sum _{n_B=0}^{s_B+a} \left( \frac{\lambda_{A1}^{n_A}}{n_A!}e^{-\lambda ^{A1}} \right) * \left( \frac{\lambda_{A2}^{S_A^{'}-S_A-a+n_A}}{(S_A^{'}-S_A-a+n_A)!}e^{-\lambda ^{A2}} \right)* \left( \frac{\lambda_{B1}^{n_B}}{n_B!}e^{-\lambda ^{B1}} \right)* \left( \frac{\lambda_{A2}^{S_B^{'}-S_B-a+n_B}}{(S_B^{'}-S_B-a+n_B)!}e^{-\lambda ^{2}} \right) 
\end{align}
&emsp;&emsp;（4）立即奖赏：该问题的立即奖赏函数为：$r([s_A,s_B],a,[s^{'}=s^{'}_A,s^{'}_B])$，即在当前状态$s=[s_A,s_B]$下，采取动作 a，到达下一状态$s_{'}=[s^{'}=s^{'}_A,s^{'}_B]$得到的立即奖赏。<br>
&emsp;&emsp;（5）折扣因子：$\gamma=0.9$。<br>
&emsp;&emsp;该问题需要从一个确定策略出发进行策略迭代。本实验的初始策略为：在任何状态下，不考虑两个租赁场的需求，选择不移动车辆（动作为 0）。然后进行策略评估，并根据值函数进行策略改进。如此反复，直到策略收敛，找到最优策略。

In [8]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson
plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签

matplotlib.use('Agg')

# maximum # of cars in each location
MAX_CARS = 20

# maximum # of cars to move during night
MAX_MOVE_OF_CARS = 5

# expectation for rental requests in first location
RENTAL_REQUEST_FIRST_LOC = 3

# expectation for rental requests in second location
RENTAL_REQUEST_SECOND_LOC = 4

# expectation for # of cars returned in first location
RETURNS_FIRST_LOC = 3

# expectation for # of cars returned in second location
RETURNS_SECOND_LOC = 2

DISCOUNT = 0.9

# credit earned by a car
RENTAL_CREDIT = 10

# cost of moving a car
MOVE_CAR_COST = 2

# all possible actions
actions = np.arange(-MAX_MOVE_OF_CARS, MAX_MOVE_OF_CARS + 1)

# An up bound for poisson distribution
# If n is greater than this value, then the probability of getting n is truncated to 0
POISSON_UPPER_BOUND = 11

# Probability for poisson distribution
# @lam: lambda should be less than 10 for this function
poisson_cache = dict()


def poisson_probability(n, lam):
    global poisson_cache
    key = n * 10 + lam
    if key not in poisson_cache:
        poisson_cache[key] = poisson.pmf(n, lam)
    return poisson_cache[key]


def expected_return(state, action, state_value, constant_returned_cars):
    """
    @state: [# of cars in first location, # of cars in second location]
    @action: positive if moving cars from first location to second location,
            negative if moving cars from second location to first location
    @stateValue: state value matrix
    @constant_returned_cars:  if set True, model is simplified such that
    the # of cars returned in daytime becomes constant
    rather than a random value from poisson distribution, which will reduce calculation time
    and leave the optimal policy/value state matrix almost the same
    """
    # initailize total return
    returns = 0.0

    # cost for moving cars
    returns -= MOVE_CAR_COST * abs(action)

    # moving cars
    NUM_OF_CARS_FIRST_LOC = min(state[0] + action, MAX_CARS)
    NUM_OF_CARS_SECOND_LOC = min(state[1] - action, MAX_CARS)

    # go through all possible rental requests
    for rental_request_first_loc in range(POISSON_UPPER_BOUND):
        for rental_request_second_loc in range(POISSON_UPPER_BOUND):
            # probability for current combination of rental requests
            prob = poisson_probability(rental_request_first_loc, RENTAL_REQUEST_FIRST_LOC) * \
                poisson_probability(rental_request_second_loc, RENTAL_REQUEST_SECOND_LOC)

            num_of_cars_first_loc = NUM_OF_CARS_FIRST_LOC
            num_of_cars_second_loc = NUM_OF_CARS_SECOND_LOC

            # valid rental requests should be less than actual # of cars
            valid_rental_first_loc = min(num_of_cars_first_loc, rental_request_first_loc)
            valid_rental_second_loc = min(num_of_cars_second_loc, rental_request_second_loc)

            # get credits for renting
            reward = (valid_rental_first_loc + valid_rental_second_loc) * RENTAL_CREDIT
            num_of_cars_first_loc -= valid_rental_first_loc
            num_of_cars_second_loc -= valid_rental_second_loc

            if constant_returned_cars:
                # get returned cars, those cars can be used for renting tomorrow
                returned_cars_first_loc = RETURNS_FIRST_LOC
                returned_cars_second_loc = RETURNS_SECOND_LOC
                num_of_cars_first_loc = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                num_of_cars_second_loc = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                returns += prob * (reward + DISCOUNT * state_value[num_of_cars_first_loc, num_of_cars_second_loc])
            else:
                for returned_cars_first_loc in range(POISSON_UPPER_BOUND):
                    for returned_cars_second_loc in range(POISSON_UPPER_BOUND):
                        prob_return = poisson_probability(
                            returned_cars_first_loc, RETURNS_FIRST_LOC) * poisson_probability(returned_cars_second_loc, RETURNS_SECOND_LOC)
                        num_of_cars_first_loc_ = min(num_of_cars_first_loc + returned_cars_first_loc, MAX_CARS)
                        num_of_cars_second_loc_ = min(num_of_cars_second_loc + returned_cars_second_loc, MAX_CARS)
                        prob_ = prob_return * prob
                        returns += prob_ * (reward + DISCOUNT *
                                            state_value[num_of_cars_first_loc_, num_of_cars_second_loc_])
    return returns


def render_car(constant_returned_cars=True):
    value = np.zeros((MAX_CARS + 1, MAX_CARS + 1))
    policy = np.zeros(value.shape, dtype=np.int)

    iterations = 0
    _, axes = plt.subplots(2, 3, figsize=(40, 20))
    plt.subplots_adjust(wspace=0.1, hspace=0.2)
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 显示中文标签
    axes = axes.flatten()
    while True:
        fig = sns.heatmap(np.flipud(policy), cmap="YlGnBu", ax=axes[iterations])
        fig.set_ylabel('# cars at first location', fontsize=30)
        fig.set_yticks(list(reversed(range(MAX_CARS + 1))))
        fig.set_xlabel('# cars at second location', fontsize=30)
        fig.set_title('policy {}'.format(iterations), fontsize=30)

        # policy evaluation (in-place)
        while True:
            old_value = value.copy()
            for i in range(MAX_CARS + 1):
                for j in range(MAX_CARS + 1):
                    new_state_value = expected_return([i, j], policy[i, j], value, constant_returned_cars)
                    value[i, j] = new_state_value
            max_value_change = abs(old_value - value).max()
            print('max value change {}'.format(max_value_change))
            if max_value_change < 1e-4:
                break

        # policy improvement
        policy_stable = True
        for i in range(MAX_CARS + 1):
            for j in range(MAX_CARS + 1):
                old_action = policy[i, j]
                action_returns = []
                for action in actions:
                    if (0 <= action <= i) or (-j <= action <= 0):
                        action_returns.append(expected_return([i, j], action, value, constant_returned_cars))
                    else:
                        action_returns.append(-np.inf)
                new_action = actions[np.argmax(action_returns)]
                policy[i, j] = new_action
                if policy_stable and old_action != new_action:
                    policy_stable = False
        print('policy stable {}'.format(policy_stable))

        if policy_stable:
            break

        iterations += 1
    plt.close()


if __name__ == '__main__':
    render_car()

max value change 196.62783361783852
max value change 134.98823859766583
max value change 91.41415360228919
max value change 67.17097732555729
max value change 51.29055484635097
max value change 38.49091000659837
max value change 29.406139835126424
max value change 25.7210573245398
max value change 22.381602293031023
max value change 19.40385808254939
max value change 16.77577350573091
max value change 14.47251552455765
max value change 12.464101852186843
max value change 10.719367983418692
max value change 9.20806226246873
max value change 7.9019189666795455
max value change 6.775146571130392
max value change 5.8045764710083745
max value change 4.969618520007145
max value change 4.252112693842776
max value change 3.6361309524054946
max value change 3.107761240497666
max value change 2.654891834022692
max value change 2.26700589940549
max value change 1.9349911763441128
max value change 1.650966802154585
max value change 1.4081276418079938
max value change 1.2006055672075036
max value c

max value change 0.0008482386133437103
max value change 0.00044346004142425954
max value change 0.00022586233239962894
max value change 0.00011383611263227067
max value change 5.771942301180388e-05
policy stable True


## 4.4 小结

&emsp;&emsp;在环境已知的前提下，基于马尔可夫决策过程，动态规划可以很好的完成强化学习任务。策略评估通常对于给定的策略，不断迭代计算每个状态（或状态-动作对）的价值。其迭代方法主要是利用对后继状态（或状态-动作对）价值的估计，来更新当前状态（或状态-动作对）价值的估计，也就是用自举的方法。策略改进是采用贪心算法，利用动作值函数获得更优的策略，每次都选择最好的动作。策略迭代是重复策略评估和策略改进的迭代，直到策略
收敛，找到最优的策略。但是策略迭代需要多次使用策略评估才能得到收敛的状态（或状态-动作对）值函数，即策略评估是迭代进行的，只有在$v_\pi$收敛时，才能停止迭代。值迭代无需等到其完全收敛，提早的计算出贪心策略，截断策略评估，在一次遍历后即刻停止策略评估，并对每个状态进行更新。实践证明，值迭代算法收敛速度优于策略迭代算法。广义策略迭代则体现了策略评估与策略改进交替进行的一般性。在GPI中，策略评估和策略改进同时进行，只要这两个过程都能不断地更新，就能收敛到最优值函数和最优策略。从这一角度看，值迭代也属于GPI，而实际上几乎所有的强化学习方法都可以被描述为 GPI。